In [ ]:
!pip install pydicom
import pydicom
import numpy as np
import os
import cv2

     |████████████████████████████████| 1.9MB 8.1MB/s 


In [ ]:
def generate_bbs(labels):
    l = []
    for label in labels:
      y,x = label.nonzero()

      ROI = np.zeros_like(label)
      if len(y) and len(x):
        ROI[y.min()-5:y.max()+5, x.min()-5:x.max()+5] = 1
      l.append(ROI)

    return np.array(l)

In [ ]:
def downsample(vid, factor=4):

  l = []
  new_shape = (vid.shape[2]//factor, vid.shape[1]//factor)
  for im in vid:
    l.append(cv2.resize(im, new_shape))

  return np.array(l)

In [ ]:
def read_ims(data_path, start, stop):
    for i in range(start,stop):
        vid_path = data_path + '/data/D' + f'{i:02}' + '.dcm'
        label_vid_path = data_path + '/groundtruth/D' + f'{i:02}' + '.dcm'

        ims_dcm = pydicom.dcmread(vid_path)
        labels_dcm = pydicom.dcmread(label_vid_path)

        ims = ims_dcm.pixel_array
        out_labels = labels_dcm.pixel_array

        in1 = downsample(ims, 4)
        out1 = downsample(out_labels, 4)

        in2 = generate_bbs(out_labels)*ims

        np.save(data_path + '/npdata/%d_in1.npy'%(i), in1)
        np.save(data_path + '/npdata/%d_out1.npy'%(i), out1)
        np.save(data_path + '/npdata/%d_in2.npy'%(i), in2)
        np.save(data_path + '/npdata/%d_out2.npy'%(i), out_labels)

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
path = './drive/My Drive/Deep_US'
start = 1
stop = 17
read_ims(path, start, stop)